### Set TACC account credentials for authenticating to the Tapis TACC Tenant 

In [ ]:
import getpass
permitted_username = 'testuser2'
permitted_user_password = 'testuser2'
# permitted_username = getpass.getpass(prompt='Username: ', stream=None)
# permitted_user_password = getpass.getpass(prompt='Password: ', stream=None)

#Set Tapis Tenant and Base URL
tenant="dev"
base_url = 'https://' + tenant + '.develop.tapis.io'

#Load Python SDK
from tapipy.tapis import Tapis

#Create python Tapis client for user
permitted_client = Tapis(base_url= base_url, 
                         username=permitted_username,
                         password=permitted_user_password, 
                         account_type='user', 
                         tenant_id=tenant
                        ) 


#Generate an Access Token that will be used for all API calls
permitted_client.get_tokens()

permitted_client.access_token

### Listing All Existing Projects
- To check if streams api is working correctly

In [ ]:
permitted_client.streams.list_projects()

### Setup Streams Variables that are used in the rest of the notebook

In [ ]:
import datetime
# storage_id = "dev.tapisv3.storage.system"
project_id = 'Mesonet' + str(datetime.datetime.today().isoformat())
site_id = 'ParkHQ'
instrument_id = '0151_ParkHQ'

### Creating the MesoNet Project

In [ ]:
result = permitted_client.streams.create_project(project_name=project_id,
                                                        description='TEST project for MesoNet',
                                                        owner='testuser2', pi='wongy', 
                                                        funding_resource='test', 
                                                        project_url='https://www.hawaii.edu/climate-data-portal/',
                                                        active=True,_tapis_debug=True)

result


In [ ]:
# list_proj = permitted_client.streams.list_projects()

# for project in list_proj:
#     delete_id = getattr(project, 'project_id')
#     permitted_client.streams.delete_project(project_id=delete_id)

### Creating the Site

In [ ]:
## Create a Streams Site
result, debug = permitted_client.streams.create_site(project_id=project_id,
                                                     request_body=[{
                                                     "site_name":site_id, 
                                                     "site_id":site_id,
                                                     "latitude":50, 
                                                     "longitude":10, 
                                                     "elevation":2,
                                                     "description":'test_site'
                                                    }], _tapis_debug=True)
print(result)

### Creating the Instrument

In [ ]:
result, debug = permitted_client.streams.create_instrument(project_id=project_id,
                                                           site_id=site_id,
                                                           request_body=[{
                                                            "inst_name":instrument_id,
                                                            "inst_description": "0151_ParkHQ",
                                                            "inst_id":instrument_id
                                                           }], _tapis_debug=True)

print(result)

### Parsing Variable Names from ikewai MesoNet file

In [ ]:
import urllib
from datetime import date, timedelta, datetime
from dateutil.rrule import rrule, DAILY

# start_date = date(2023, 1, 1)
# end_date = datetime.today().date()

# for dt in rrule(DAILY, dtstart=start_date, until=end_date):
#     curr_date = dt.date()

#     files = [
#         "0115_Piiholo_MetData.dat",
#         "0116_Keokea_MetData.dat",
#         "0119_KulaAg_MetData.dat",
#         "0143_Nakula_MetData.dat",
#         "0151_ParkHQ_MetData.dat",
#         "0152_NeneNest_MetData.dat",
#         "0153_Summit_MetData.dat",
#         "0281_IPIF_MetData.dat",
#         "0282_Spencer_MetData.dat",
#         "0283_Laupahoehoe_MetData.dat",
#         "0286_Palamanui_MetData.dat",
#         "0287_Mamalahoa_MetData.dat",
#         "0501_Lyon_MetData_5min.dat",
#         "0502_NuuanuRes1_MetData.dat",
#         "0601_Waipa_MetData.dat",
#         "0602_CommonGround_MetData.dat"
#     ]
#     base_url = "https://ikeauth.its.hawaii.edu/files/v2/download/public/system/ikewai-annotated-data/HCDP/raw/"

#     year = curr_date.year
#     month = curr_date.month
#     day = curr_date.day

#     for file in files:
#         link = f"{base_url}{year}/{month}/{day}/{file}"

    
link = 'https://ikeauth.its.hawaii.edu/files/v2/download/public/system/ikewai-annotated-data/HCDP/raw/2022/10/25/0151_ParkHQ_MetData.dat'

data_file = urllib.request.urlopen(link).readlines()

list_vars = data_file[1].decode("UTF-8").strip().replace("\"", "").split(",")

list_units = data_file[2].decode("UTF-8").strip().replace("\"", "").split(",")

print(list_vars, list_units)

### Creating the Variables

In [ ]:
request_body = []
for i in range(2, len(list_vars)):
    request_body.append({
        "var_id": list_vars[i],
        "var_name": list_vars[i],
        "units": list_units[i]
    })

#Create variables in bulk
result, debug = permitted_client.streams.create_variable(project_id=project_id,
                                                         site_id=site_id,
                                                         inst_id=instrument_id,
                                                         request_body=request_body,_tapis_debug=True)

print(result)

### Parsing Measurements from MesoNet file

In [ ]:

variables = []

for i in range(4, len(data_file)):
    measurements = data_file[i].decode("UTF-8").strip().replace("\"", "").split(",")
    measurement = {}
    time = measurements[0].split(" ")
    if(int(time[1].split(":")[0]) > 23):
        time_string = time[0] + " 23:59:59"
        time_string = datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S')
        time_string += timedelta(seconds=1)
    else:
        time_string = datetime.strptime(measurements[0], '%Y-%m-%d %H:%M:%S')
    measurement['datetime'] = time_string.isoformat()+"-10:00"
    for j in range(2, len(measurements)):
        measurement[list_vars[j]] = float(measurements[j])

    variables.append(measurement)

result = permitted_client.streams.create_measurement(inst_id=instrument_id,
                                                            vars=variables)

print(result)


In [ ]:
print(data_file[5].decode("UTF-8").strip().replace("\"", "").split(","))